In [17]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated, operator
from dotenv import load_dotenv
from pydantic import BaseModel, Field

In [18]:
load_dotenv()

True

In [19]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [20]:
class EssayEvaluationSchema(BaseModel):
    feedback: str = Field(description="detailed feedback on the essay")
    score: int = Field(description="score for the essay (0-10)", ge=0, le=10)

# class EssayEvaluationOutput(BaseModel):
#     grammar_score: int = Field(..., description="Score for grammar (0-10)")
#     coherence_score: int = Field(..., description="Score for coherence (0-10)")
#     relevance_score: int = Field(..., description="Score for relevance (0-10)")
#     overall_score: int = Field(..., description="Overall score (0-10)")

In [21]:
structured_model = model.with_structured_output(EssayEvaluationSchema)

In [22]:
essay = """ ## The Role of Artificial Intelligence in Business

Artificial Intelligence, commonly known as AI, is transforming how businesses operate, compete, and grow. It refers to computer systems that can perform tasks typically requiring human intelligence, such as learning from data, recognizing patterns, understanding language, and making decisions. In modern business environments, AI is no longer optional but increasingly a strategic necessity.

One of the most important roles of AI in business is improving decision making. According to a report by McKinsey & Company, organizations that effectively use AI-driven analytics can significantly enhance performance by extracting insights from large volumes of data. Machine learning models can analyze customer behavior, forecast demand, detect anomalies, and support data-driven strategies that reduce uncertainty and risk.

AI also plays a major role in operational efficiency. Technologies such as robotic process automation automate repetitive administrative tasks like data entry, invoice processing, and report generation. This reduces human error and allows employees to focus on higher value activities. According to Deloitte, intelligent automation can improve productivity while lowering operational costs when implemented strategically.

Customer experience is another area where AI delivers strong impact. AI powered chatbots and virtual assistants provide 24 hour support, personalized recommendations, and faster response times. Research from IBM shows that AI chatbots can handle a large percentage of routine customer inquiries, helping companies reduce service costs while maintaining service quality.

In marketing and sales, AI enables predictive targeting and personalization. Algorithms analyze browsing history, purchase behavior, and engagement patterns to deliver tailored advertisements and product suggestions. According to PwC, AI is expected to contribute significantly to global economic growth through productivity gains and consumer personalization.

However, the adoption of AI also raises ethical and governance concerns. Businesses must ensure data privacy, transparency, and fairness in algorithmic decisions. Regulatory frameworks such as the EU AI Act are emerging to guide responsible AI deployment. Companies that balance innovation with ethical responsibility are more likely to build long term trust.

In conclusion, AI plays a transformative role in business by enhancing decision making, increasing efficiency, improving customer experience, and driving innovation. As AI technologies continue to evolve, businesses that strategically integrate them while maintaining ethical standards will gain a competitive advantage in the global marketplace.
"""

In [23]:
prompt = f"Evaluate the following essay and provide detailed feedback along with a score from 0 to 10:\n\n{essay}"
result = structured_model.invoke(prompt)
print(result.score)
print(result.feedback)


7
This essay provides a comprehensive overview of the role of Artificial Intelligence in business, touching on key areas such as decision making, operational efficiency, customer experience, and marketing. The structure is clear and logical, moving from a general introduction to specific applications and considerations, concluding with a strong summary that encapsulates the main points.

The introduction effectively sets the stage for discussing AI's impact, and the use of statistics and references (e.g., McKinsey & Company, Deloitte, IBM, PwC) adds credibility to the claims made throughout the essay. However, it would benefit from a few improvements:

1. **Depth of Analysis**: While the essay covers various applications of AI, it lacks depth in discussing how these technologies are implemented in practice. Including specific examples of companies or case studies could enhance the argument and provide real-world context.

2. **Ethical Considerations**: The discussion of ethical concern

/Users/junaeidshoaib/Documents/LangGraph/myenv/lib/python3.11/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='parsed', input_value=EssayEvaluationSchema(fee...implications.", score=7), input_type=EssayEvaluationSchema])
  return self.__pydantic_serializer__.to_python(


In [24]:
class EvalutionState(TypedDict):

    essay: str
    language_feedback: str
    clarity_feedback: str
    analysis_feedback: str
    overall_feedback: str

    individual_scores: Annotated[list[int], operator.add]
    overall_score: float

In [25]:
def evaluate_language(state: EvalutionState) -> EvalutionState:
    prompt = f"Evaluate the language quality used in the following essay and provide detailed feedback along with a score from 0 to 10:\n\n{state['essay']}"
    result = structured_model.invoke(prompt)
    return {'language_feedback': result.feedback, 'individual_scores': [result.score]}

def evaluate_clarity(state: EvalutionState) -> EvalutionState:
    prompt = f"Evaluate the clarity of the following essay and provide detailed feedback along with a score from 0 to 10:\n\n{state['essay']}"
    result = structured_model.invoke(prompt)
    return {'clarity_feedback': result.feedback, 'individual_scores': [result.score]}

def evaluate_analysis(state: EvalutionState) -> EvalutionState:
    prompt = f"Evaluate the depth of analysis in the following essay and provide detailed feedback along with a score from 0 to 10:\n\n{state['essay']}"
    result = structured_model.invoke(prompt)
    return {'analysis_feedback': result.feedback, 'individual_scores': [result.score]}

def final_evaluation(state: EvalutionState) -> EvalutionState:
    prompt = f"Based on the following feedback, provide an overall summarised evaluation of the essay along with a final score from 0 to 10:\n\nLanguage Feedback: {state['language_feedback']}\nClarity Feedback: {state['clarity_feedback']}\nAnalysis Feedback: {state['analysis_feedback']}"
    result = model.invoke(prompt).content
    avg_score = sum(state['individual_scores']) / len(state['individual_scores'])
    return {'overall_feedback': result, 'overall_score': avg_score}

In [27]:
graph = StateGraph(EvalutionState)

graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_clarity', evaluate_clarity)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('final_evaluation', final_evaluation)

#edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_clarity')
graph.add_edge(START, 'evaluate_analysis')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_clarity', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('final_evaluation', END)

workflow = graph.compile()

from IPython.display import display
display(workflow)

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`